## **Data Collection**

### **1. Scraping Web**

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import requests
import re
import pandas as pd
import numpy as np

In [ ]:
def download_html(url):
    with urllib.request.urlopen(url) as response:
        html = response.read()
        html = html.decode('utf-8')
    response.close()
    return html

Hàm ***get_house_details***  lấy dữ liệu:  mã bài đăng; thời gian đăng bài; tên bài đăng; nội dung bài đăng; diện tích nhà; số phòng ngủ; số phòng toilet; hướng nhà; loại nhà; địa chỉ nhà: tỉnh, huyện; giá nhà.

In [ ]:
def get_house_details(link):
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')
    try:
        detail = soup.find('h1',{'class':'uk-panel-title'}).text #find detail
    except:
        detail = None
    try:
        acreage = float(soup.find('strong',string='Diện tích:').next_sibling[:-1].strip()) #find acreage
    except:
        acreage = None
    try:
        bedroom = int(soup.find('strong',string='Phòng ngủ:').next_sibling.split()[0]) #find number of bedroom
    except:
        bedroom = None
    try:
        bathroom = int(soup.find('strong',string='Phòng WC:').next_sibling.split()[0]) #find number of bathroom
    except:
        bathroom = None
    try:
        direction = soup.find('strong',string='Hướng nhà:').next_sibling.strip() #find house's direction
    except:
        direction = None
    try:
        content = soup.find('div',{'class':'content'}).text.strip() #find content
    except:
        content = None
    try:
        house_type=soup.find('ul',{'class':'uk-breadcrumb'}).find_all('li')[1].text #find type of house
    except:
        house_type = None
    try:
        datetime=soup.find('time',{'class':'timeago'}).get('datetime') #find datatime
    except:
        datetime = None
    try:
        city = soup.find('ul',{'class':'uk-breadcrumb'}).find_all('li')[2].text #find city
    except:
        city = None
    try:
        district = soup.find('ul',{'class':'uk-breadcrumb'}).find_all('li')[3].text #find district
    except:
        district = None
    try:
        price = soup.find('strong',{'class':'price'}).text.strip() #find price
    except:
        price = None
    try:
        id = soup.find('strong',string='Mã tin:').next_sibling.text.strip() #find id
    except:
        id = None

    return id, detail, acreage, bedroom, bathroom, direction, content, house_type, datetime, city, district, price

In [ ]:
def house_details(url, houses):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    for i in soup.find_all('div',{'class':'body'})[:-5]:
        link = i.find('a')['href']
        house_info = get_house_details(link)
        houses.append(house_info)

In [ ]:
url = 'https://batdongsan.vn/ban-nha/'
link = []
for i in range(2,481):
    link.append('https://batdongsan.vn/ban-nha/p' + str(i))
houses = []
house_details(url, houses)
for i in link:
    house_details(i, houses)

In [ ]:
houses_data = pd.DataFrame(houses, columns = ['Id','Detail', 'Acreage','TotalBedroom','TotalBathroom','Direction','Content','HouseType','Datetime','City', 'District','Price'])
houses_data

> Giải thích tên các cột trong dataframe **'houses_data'**

- *Id*: mã tin
- *Detail*: Nội dung chi tiết bài đăng
- *Acreage*: Diện tích nhà
- *TotalBedroom*: Tổng số phòng ngủ
- *TotalBathroom*: Tổng số phòng tắm
- *Direction*: Hướng nhà
- *Content*: Tên bài đăng
- *HouseType*: Loại nhà
- *Datetime*: Thời gian đăng bài
- *City*: Tỉnh/Thành phó
- *District*: Quận/Huyện
- *Price*: Giá bán nhà

Lưu data frame thành file csv với tên là ***Dataset.csv***

In [ ]:
import csv
houses_data.to_csv('Dataset.csv', encoding='utf-8', index=False)

### **2. Features Sellection**

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
file=pd.read_csv('Dataset.csv')
file.head()

,Id,Detail,Acreage,TotalBedroom,TotalBathroom,Direction,Content,HouseType,Datetime,City,District,Price
0,268184.0,"Bán Biệt thự sân vườn Quận 12, 130m2, 3 Tầng 7...",130.0,NaN,NaN,NaN,"+ Kết cấu: 7 PN, 6 WC, ban công, phòng thờ, Ph...",Bán Nhà riêng,2023-05-29 13:06:27,TP Hồ Chí Minh,Quận 12,9.2 tỷ
1,268704.0,2.9 Tỷ Diện Tích 65/80m2 Nhà Kiệt Lê Văn Hưu N...,65.0,NaN,NaN,NaN,2.9 Tỷ Diện Tích 65/80m2 Nhà Kiệt Lê Văn Hưu N...,Bán Nhà riêng,2023-06-05 15:16:48,Đà Nẵng,Ngũ Hành Sơn,2.9 tỷ
2,268701.0,Bán Nhà Kim Ngưu – Phân Lô Ô Tô – Nở Hậu – Mặt...,90.0,5.0,4.0,NaN,“KHÔNG MUA NHÀ NÀY THÌ MUA NHÀ NÀO”\n\r\n- BÁN...,Bán Nhà riêng,2023-06-05 10:35:38,Hà Nội,Hai Bà Trưng,14 tỷ
3,268700.0,Quận 3 - Lê Văn Sỹ - 5 TẦNG BTCT - Hẻm xe hơi ...,NaN,NaN,NaN,NaN,+ Kế bên Quận 1 - Quận 3 -Khu dân cư sầm uất \...,Bán Nhà riêng,2023-06-05 10:38:45,TP Hồ Chí Minh,Quận 3,Thỏa thuận
4,268686.0,NHÀ MỚI - FULL NỘI THẤT - HXH TỚI NHÀ - 20M RA...,30.0,2.0,2.0,NaN,"Mô tả\r\n+ Kết cấu: 1 trệt 1 lầu, nhà mới cứng...",Bán Nhà riêng,2023-06-05 12:18:18,TP Hồ Chí Minh,Gò Vấp,3.7 tỷ


In [ ]:
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9600 entries, 0 to 9599
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             9596 non-null   float64
 1   Detail         9596 non-null   object 
 2   Acreage        8797 non-null   float64
 3   TotalBedroom   5967 non-null   float64
 4   TotalBathroom  5497 non-null   float64
 5   Direction      1186 non-null   object 
 6   Content        9600 non-null   object 
 7   HouseType      9596 non-null   object 
 8   Datetime       9600 non-null   object 
 9   City           9596 non-null   object 
 10  District       9596 non-null   object 
 11  Price          9596 non-null   object 
dtypes: float64(4), object(8)
memory usage: 900.1+ KB


Nhận thấy bộ dữ liệu trên là chưa đủ, thiếu một số tính năng, và những tính năng đó có thể thu thập được từ cột *Detail* và *Content* của dataframe. Sử dụng regex để thực hiện

Tìm thêm dữ liệu cho 'TotalBedroom', 'TotalBathroom', 'Acreage', 'Direction'.
Tạo thêm thuộc tính mới *HouseUtilities*, *HouseMinusPoint*, *RoomMultipurpose*,*OuterUtilities*, *HouseType*, *TotalFloor*

In [ ]:
list_total_bedroom=file['TotalBedroom'].values
list_content=file['Content'].values
list_total_bathroom=file['TotalBathroom'].values
list_direction=file['Direction'].values
list_acreage=file['Acreage'].astype(str)
list_detail=file['Detail'].values

In [ ]:
list_total_floor=[]
#lấy tổng số tầng, hầm
regex1=re.compile(r'(\d+).?(tầng|lầu|trệt)')
for i in range(len(list_content)):
    list_floor= regex1.findall(list_content[i].lower())
    total_floor=0
    for j in range(len(list_floor)):
        total_floor+=int(list_floor[j][0])
    if total_floor==0:
        total_floor=1
    list_total_floor.append(total_floor)

In [ ]:
#lấy số tiện ích của căn nhà
house_utilities=[]
regex2=re.compile('rộng|tốt|nội thất|thang máy|nở hậu|đẹp|thoáng|hiện đại|công năng|mới|vuông|sổ riêng|to')

In [ ]:
#lấy số phòng có công năng thêm
room_multipurpose=[]
regex3=re.compile('phòng thờ|phòng giặt|phòng đọc sách|garage|sân thượng|sân phơi|gara|ban công|sân|kho|sân vườn')

In [ ]:
#lấy số tiện ích bên ngoài
outer_utilities=[]
regex4=re.compile('mặt tiền|đường lớn|phố|trung tâm|giao thông|an ninh|thuận tiện|vỉa hè|2 ngõ|thông ngõ|dân trí|an ninh|bảo vệ|sạch|trường học|siêu thị|bệnh viện|đại học|gần|xe')

In [ ]:
#tính tổng số các điểm trừ trong nhà
regex5=re.compile('hẻm nhỏ|ngõ cụt|góc|ngắn|thấp|sổ chung')
minus_point=[]
for i in range(len(list_content)):
    num_house_utilitie=len(regex2.findall(list_content[i].lower()))
    num_room_multipurpose=len(regex3.findall(list_content[i].lower()))
    num_outer_utilitie=len(regex4.findall(list_content[i].lower()))
    num_minus_point=len(regex5.findall(list_content[i].lower()))

    house_utilities.append(num_house_utilitie)
    room_multipurpose.append(num_room_multipurpose)
    outer_utilities.append(num_outer_utilitie)
    minus_point.append(num_minus_point)

In [ ]:
#lấy số phòng ngủ
regex6=re.compile(r'(\d+).?(phòng ngủ|ngủ|pngu|pn|master)')
for i in range(len(list_content)):
    if pd.isnull(list_total_bedroom[i]):
        list_bedroom=regex6.findall(list_content[i].lower())
        num_bedroom=0
        for j in list_bedroom:
            num_bedroom+=int(j[0])
        if num_bedroom==0:
            num_bedroom=np.nan
        list_total_bedroom[i]=num_bedroom
file['TotalBedroom']=list_total_bedroom

In [ ]:
#lấy số phòng wc
regex7=re.compile(r'(\d+).?(wc|toilet|vs|nhà vệ sinh|phòng tắm)')
for i in range(len(list_content)):
    if pd.isnull(list_total_bathroom[i]):
        list_bathroom=regex7.findall(list_content[i].lower())
        num_bathroom=0
        for j in list_bathroom:
            num_bathroom+=int(j[0])
        if(num_bathroom==0):
            num_bathroom=np.nan
        list_total_bathroom[i]=num_bathroom
file['TotalBathroom']=list_total_bathroom

In [ ]:
#tìm hướng nhà
regex8=re.compile('đông.?nam|đông.?bắc|tây.?nam|tây.?bắc|đông|tây|nam|bắc')
for i in range(len(list_content)):
    if pd.isnull(list_direction[i]):
        direction=regex8.findall(list_content[i].lower())
        if direction==[]:
            direction = None
        else:
            direction=str.title(direction[0])
file['Direction']=list_direction

In [ ]:
#lấy diện tích còn thiếu
regex9=re.compile(r'(\d+).?(m2)')
for i in range(len(list_content)):
    if pd.isnull(list_acreage[i]):
        acreage = regex9.findall(list_detail[i])
        if acreage==[]:
            acreage=regex9.findall(list_content[i])
            if acreage==[]:
                list_acreage[i]=np.nan
            else:
                list_acreage[i]=float(acreage[0][0])
        else:
            list_acreage[i]=float(acreage[0][0])

file['Acreage']=list_acreage

In [ ]:
file['TotalFloor']=list_total_floor
file['HouseUtilities']=house_utilities
file['OuterUtilities']=outer_utilities
file['RoomMultipurpose']=room_multipurpose
file['HouseMinusPoint']=minus_point

Xoá những hàng không có thông tin mô tả chi tiết.

In [ ]:
print(file.shape)
file=file.dropna(subset=['Detail'])
print(file.shape)

(9600, 17)
(9596, 17)


Sau khi lấy được các dữ liệu cần thiết từ 2 cột *Detail* và *Content*, ta drop những cột không liên quan đến số liệu như: *Id*, *Detail*, *Content* để dễ dàng xử lí dữ liệu.

In [ ]:
file=file.drop(['Detail','Content','Id'],axis=1)
file=file[['Datetime','Acreage','TotalFloor','TotalBedroom','TotalBathroom','HouseUtilities','HouseMinusPoint','RoomMultipurpose','OuterUtilities','HouseType','Direction','City','District','Price']]
file

,Datetime,Acreage,TotalFloor,TotalBedroom,TotalBathroom,HouseUtilities,HouseMinusPoint,RoomMultipurpose,OuterUtilities,HouseType,Direction,City,District,Price
0,2023-05-29 13:06:27,130.0,1,7.0,6.0,4,0,4,4,Bán Nhà riêng,NaN,TP Hồ Chí Minh,Quận 12,9.2 tỷ
1,2023-06-05 15:16:48,65.0,1,2.0,NaN,2,0,1,10,Bán Nhà riêng,NaN,Đà Nẵng,Ngũ Hành Sơn,2.9 tỷ
2,2023-06-05 10:35:38,90.0,10,5.0,4.0,5,0,4,10,Bán Nhà riêng,NaN,Hà Nội,Hai Bà Trưng,14 tỷ
3,2023-06-05 10:38:45,nan,5,5.0,NaN,1,0,1,1,Bán Nhà riêng,NaN,TP Hồ Chí Minh,Quận 3,Thỏa thuận
4,2023-06-05 12:18:18,30.0,2,2.0,2.0,3,0,0,4,Bán Nhà riêng,NaN,TP Hồ Chí Minh,Gò Vấp,3.7 tỷ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9595,2022-10-26 16:35:07,83.0,1,5.0,5.0,2,1,4,6,Bán Nhà riêng,Đông,TP Hồ Chí Minh,Gò Vấp,54444.44 tỷ
9596,2022-10-26 16:17:07,208.0,1,3.0,2.0,1,0,0,2,Bán Nhà mặt phố,NaN,TP Hồ Chí Minh,Quận 9,10.8 tỷ
9597,2022-10-26 15:57:58,54.0,3,3.0,3.0,3,0,1,6,Bán Nhà riêng,NaN,TP Hồ Chí Minh,Bình Thạnh,5.5 tỷ
9598,2022-10-26 15:48:12,110.0,1,2.0,1.0,5,0,0,15,Bán Nhà mặt phố,NaN,Ninh Thuận,Ninh Sơn,5.5 tỷ


Lưu data frame thành file csv với tên là ***Dataset_house_price_new.csv***

In [ ]:
file.to_csv('Dataset_house_price.csv', encoding='utf-8', index=False)